# 1 Download and Explore Dataset

In [36]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

Scrape the Wikipedia page and wrangle the data

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

Check the length of the first 10 rows

In [3]:
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content().strip()
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood


In [5]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content().strip()
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [6]:
[len(C) for (title,C) in col]

[287, 287, 287]

Transform the data into a pandas dataframe

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ignore cells with a borough that is Not assigned

In [9]:
df = df[df.Borough != "Not assigned"]
df.reset_index(drop=True, inplace=True)

Take a quick look at the data

In [10]:
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Not assigned
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [11]:
df=df.groupby(['Postcode','Borough'], as_index=False, sort = False).agg(lambda x: ','.join(x))
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [12]:
df.loc[df['Neighbourhood'] == "Not assigned",'Neighbourhood' ] = df['Borough']

In [13]:
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [14]:
df.shape

(103, 3)

# 2 Get the latitude and the longitude coordinates of each neighborhood

import the csv file that has the geographical coordinates of each postal code

In [15]:
path="http://cocl.us/Geospatial_data"
df_ll=pd.read_csv(path)

In [16]:
df_ll=df_ll.rename(columns={"Postal Code": "Postcode", "Latitude":"Latitude", "Longitude":"Longitude"})
df_ll.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge two tables

In [17]:
df_geo = pd.merge(df, df_ll, on='Postcode', how='outer')
df_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# 3 Explore and cluster the neighborhoods in Toronto

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_geo['Borough'].unique()),
        df_geo.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [19]:
print(df_geo['Borough'].unique())

['North York' 'Downtown Toronto' "Queen's Park" 'Scarborough' 'East York'
 'Etobicoke' 'York' 'East Toronto' 'West Toronto' 'Central Toronto'
 'Mississauga']


In [20]:
#df_geo_clustering = df_geo.groupby('Borough').mean().reset_index()
#df_geo_clustering

In [24]:
df_geo_clustering = df_geo.drop('Borough', 1)
df_geo_clustering = df_geo_clustering.drop('Neighbourhood', 1)
df_geo_clustering = df_geo_clustering.drop('Postcode', 1)
df_geo_clustering.head()

,Latitude,Longitude
0,43.753259,-79.329656
1,43.725882,-79.315572
2,43.654260,-79.360636
3,43.718518,-79.464763
4,43.662301,-79.389494


In [25]:
from sklearn.cluster import KMeans

In [26]:
# set number of clusters
kclusters = 3
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_geo_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 0, 2, 0, 2, 1, 0, 1, 0], dtype=int32)

In [27]:
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



In [28]:
df_geo.insert(0, 'Cluster Labels', kmeans.labels_)
df_geo.head()

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,1,M3A,North York,Parkwoods,43.753259,-79.329656
1,1,M4A,North York,Victoria Village,43.725882,-79.315572
2,0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,0,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [29]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [30]:
address = 'Toronto City, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Neighbourhood'], df_geo['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters